## Spotify and YouTube dataset population

This notebook outlines the steps to create an RDF dataset based on the SoundGraph ontology, from the data import to RDF triple export in Turtle format.

In [1]:
# required libraries
from pathlib import Path
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import FOAF, XSD
import ast
import os
import pandas as pd
import re

### Load CSV files + preprocessing

In [2]:
# csv files path
base_path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())

dataset_path = base_path + '/data/Spotify_Youtube.csv'
spotify_artist_path = base_path + '/data/Artists.csv'
spotify_artist_info_path = base_path + '/data/Artist_info.csv'
spotify_album_path = base_path + '/data/Album_info.csv'
wikidata_artists_path = base_path + '/data/wikidata_artists.csv'
wikidata_award_statements_path = base_path + '/data/wikidata_award_statements.csv'
wikidata_awards_path = base_path + '/data/wikidata_awards_processed.csv'
youtube_api_channels_path = base_path + '/data/youtubeapi_channels_complete.csv'
# target path where to save the serializations
rdf_path = base_path + '/rdf/' 

In [3]:
dataset = pd.read_csv(dataset_path)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20718 entries, 0 to 20717
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        20718 non-null  int64  
 1   Artist            20718 non-null  object 
 2   Url_spotify       20718 non-null  object 
 3   Track             20718 non-null  object 
 4   Album             20718 non-null  object 
 5   Album_type        20718 non-null  object 
 6   Uri               20718 non-null  object 
 7   Danceability      20716 non-null  float64
 8   Energy            20716 non-null  float64
 9   Key               20716 non-null  float64
 10  Loudness          20716 non-null  float64
 11  Speechiness       20716 non-null  float64
 12  Acousticness      20716 non-null  float64
 13  Instrumentalness  20716 non-null  float64
 14  Liveness          20716 non-null  float64
 15  Valence           20716 non-null  float64
 16  Tempo             20716 non-null  float6

In [4]:
# TODO ma questo csv serve?
spotify_artist = pd.read_csv(spotify_artist_path)
spotify_artist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2079 entries, 0 to 2078
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2079 non-null   int64 
 1   Artist      2079 non-null   object
dtypes: int64(1), object(1)
memory usage: 32.6+ KB


In [5]:
spotify_artist_info = pd.read_csv(spotify_artist_info_path)
spotify_artist_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2079 entries, 0 to 2078
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Artist      2079 non-null   object
 1   Followers   2079 non-null   int64 
 2   Genres      2079 non-null   object
 3   Popularity  2079 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 65.1+ KB


In [6]:
wikidata_artists = pd.read_csv(wikidata_artists_path)
wikidata_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2079 entries, 0 to 2078
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Artist         2079 non-null   object
 1   Url_spotify    2079 non-null   object
 2   artistLabel    2079 non-null   object
 3   websiteLabel   2079 non-null   object
 4   start          2079 non-null   object
 5   end            2079 non-null   object
 6   dissolved      2079 non-null   object
 7   country_codes  2079 non-null   object
dtypes: object(8)
memory usage: 130.1+ KB


In [7]:
spotify_album = pd.read_csv(spotify_album_path)
spotify_album.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20718 entries, 0 to 20717
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                20718 non-null  object
 1   Album             20641 non-null  object
 2   Total_tracks      20718 non-null  int64 
 3   Release_date      20718 non-null  object
 4   Available_market  20718 non-null  object
dtypes: int64(1), object(4)
memory usage: 809.4+ KB


In [8]:
youtube_api_channels = pd.read_csv(youtube_api_channels_path)
youtube_api_channels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6715 entries, 0 to 6714
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   channelId           6715 non-null   object
 1   title               6715 non-null   object
 2   channelDescription  4241 non-null   object
 3   viewCount           6715 non-null   object
 4   subscriberCount     6715 non-null   object
 5   videoCount          6715 non-null   object
 6   error               6715 non-null   int64 
 7   originalChannel     6715 non-null   object
dtypes: int64(1), object(7)
memory usage: 419.8+ KB


In [9]:
wikidata_awards = pd.read_csv(wikidata_awards_path)
wikidata_awards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   award_id        193 non-null    int64 
 1   award_name      193 non-null    object
 2   award_type      193 non-null    object
 3   award_category  193 non-null    object
 4   award_class     193 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.7+ KB


In [10]:
wikidata_awards_statements = pd.read_csv(wikidata_award_statements_path)
wikidata_awards_statements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist_spotify_id  1001 non-null   object
 1   award_id           1001 non-null   int64 
 2   award_year         1001 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


### Data cleaning
We remove NaN values and make IDs explicit in the columns.

In [11]:
dataset.isnull().any()

Unnamed: 0          False
Artist              False
Url_spotify         False
Track               False
Album               False
Album_type          False
Uri                 False
Danceability         True
Energy               True
Key                  True
Loudness             True
Speechiness          True
Acousticness         True
Instrumentalness     True
Liveness             True
Valence              True
Tempo                True
Duration_ms          True
Url_youtube          True
Title                True
Channel              True
Views                True
Likes                True
Comments             True
Description          True
Licensed             True
official_video       True
Stream               True
dtype: bool

In [12]:
# fill values
numeric_fillna = -50 # Loudness is measured in dB, min value is around -46.3
string_fillna = '_'
boolean_fillna = False

In [13]:
# numeric
dataset['Danceability'] = dataset['Danceability'].fillna(numeric_fillna)
dataset['Energy'] = dataset['Energy'].fillna(numeric_fillna)
dataset['Key'] = dataset['Key'].fillna(numeric_fillna)
dataset['Loudness'] = dataset['Loudness'].fillna(numeric_fillna)
dataset['Speechiness'] = dataset['Speechiness'].fillna(numeric_fillna)
dataset['Acousticness'] = dataset['Acousticness'].fillna(numeric_fillna)
dataset['Instrumentalness'] = dataset['Instrumentalness'].fillna(numeric_fillna)
dataset['Liveness'] = dataset['Liveness'].fillna(numeric_fillna)
dataset['Valence'] = dataset['Valence'].fillna(numeric_fillna)
dataset['Tempo'] = dataset['Tempo'].fillna(numeric_fillna)
dataset['Duration_ms'] = dataset['Duration_ms'].fillna(numeric_fillna)
dataset['Views'] = dataset['Views'].fillna(numeric_fillna)
dataset['Likes'] = dataset['Likes'].fillna(numeric_fillna)
dataset['Comments'] = dataset['Comments'].fillna(numeric_fillna)
dataset['Stream'] = dataset['Stream'].fillna(numeric_fillna)
# string -> '_'
dataset['Url_youtube'] = dataset['Url_youtube'].fillna(string_fillna)
dataset['Title'] = dataset['Title'].fillna(string_fillna)
dataset['Channel'] = dataset['Channel'].fillna(string_fillna)
dataset['Description'] = dataset['Description'].fillna(string_fillna)
# bool -> False
dataset['Licensed'] = dataset['Licensed'].fillna(boolean_fillna)
dataset['official_video'] = dataset['official_video'].fillna(boolean_fillna)

In [14]:
# make IDs explicit
dataset['Url_spotify'] = dataset['Url_spotify'].apply(lambda uri: uri.split('/')[-1])
dataset['Uri'] = dataset['Uri'].apply(lambda uri: uri.split(':')[-1])
dataset['Url_youtube'] = dataset['Url_youtube'].apply(lambda uri: uri.split('?v=')[-1])

We also observed that some videos and some tracks have different values for the same column and in our dataset this has no sense (e.g, a video having two different ```views``` values: we don't work with the video over time, so this shouldn't happen).

Videos:
- views, likes, comments: we take the ```max``` value
- official_video: we set it to ```false```

Tracks:
- streams: we take the ```max``` value

In [15]:
%%time
# dataset cleaning
# some videos have more than one value for views, likes, comments and official_video
# we take the max for everything but official_video, for which we just set to false
yt_video_urls = dataset[['Url_youtube']].drop_duplicates().reset_index(drop=True)
for index, row in yt_video_urls.iterrows():
    # get rows with same youtube video URL
    videos = dataset[dataset['Url_youtube'] == row['Url_youtube']]
    # if there is more than one row
    if len(videos) > 1:
        # take max values
        max_views = videos['Views'].max()
        max_likes = videos['Likes'].max()
        max_comments = videos['Comments'].max()
        # check if official_video needs to be fixed 
        fix_official_video = False
        if len(videos['official_video'].drop_duplicates()) > 1:
            fix_official_video = True
        # fix values
        for row_idx in videos.index:
            dataset.at[row_idx, 'Views'] = max_views
            dataset.at[row_idx, 'Likes'] = max_likes
            dataset.at[row_idx, 'Comments'] = max_comments
            if fix_official_video:
                dataset.at[row_idx, 'official_video'] = False
                
# the same thing can happen with stream of spotify song: also here we take the max value
spotify_track_uris = dataset[['Uri']].drop_duplicates().reset_index(drop=True)
for index, row in spotify_track_uris.iterrows():
    # get rows with the same spotify track
    tracks = dataset[dataset['Uri'] == row['Uri']]
    # if there is more than one track
    if len(tracks) > 1:
        # take the max value and set it on all the rows
        max_streams = tracks['Stream'].max()
        for row_idx in tracks.index:
            dataset.at[row_idx, 'Stream'] = max_streams

CPU times: user 47.7 s, sys: 6.19 ms, total: 47.7 s
Wall time: 47.7 s


In [16]:
spotify_album.isnull().any()

Id                  False
Album                True
Total_tracks        False
Release_date        False
Available_market    False
dtype: bool

In [17]:
spotify_album['Id'] = spotify_album['Id'].apply(lambda uri: uri.split(':')[-1])
spotify_album['Album'] = spotify_album['Album'].fillna(string_fillna)

### rdflib setup

In [18]:
# Set the countries and the SoundGraph ontologies namespaces (not known by rdflib)
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
SG = Namespace("https://www.dei.unipd.it/db2/ontology/soundgraph#")

# create the graph
g = Graph()

# bind the namespaces to a prefix
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("sg", SG)

In [19]:
# util function to dump the graph in a file and get a new empty graph with the bindings already set
def write_and_empty_graph(graph, filename):
    with open(rdf_path + filename, 'w') as file:
        file.write(graph.serialize(format='turtle'))
        
    graph = Graph()
    graph.bind("foaf", FOAF)
    graph.bind("xsd", XSD)
    graph.bind("countries", CNS)
    graph.bind("sg", SG)
    return graph

### YouTube Video

In [20]:
%%time
for index, row in dataset.iterrows():
    if row['Url_youtube'] != string_fillna:
        video_id = 'video_' + row['Url_youtube']
        Video = URIRef(SG[video_id])
        g.add((Video, RDF.type, SG.YouTubeVideo))
        
        # data properties
        g.add((Video, SG['videoTitle'], Literal(row['Title'], datatype=XSD.string)))
        if row['Description'] != string_fillna:
            g.add((Video, SG['videoDescription'], Literal(row['Description'], datatype=XSD.string)))
        if row['Comments'] != numeric_fillna:
            g.add((Video, SG['videoComments'], Literal(row['Comments'], datatype=XSD.integer)))
        if row['Likes'] != numeric_fillna:
            g.add((Video, SG['videoLikes'], Literal(row['Likes'], datatype=XSD.integer)))
        g.add((Video, SG['videoViews'], Literal(row['Views'], datatype=XSD.integer)))
        g.add((Video, SG['isOfficialVideo'], Literal(row['official_video'], datatype=XSD.boolean)))
        g.add((Video, SG['isLicensed'], Literal(row['Licensed'], datatype=XSD.boolean)))

CPU times: user 5.61 s, sys: 290 ms, total: 5.9 s
Wall time: 5.89 s


### YouTube Channel + publishes

In [21]:
%%time
for index, row in youtube_api_channels.iterrows():
    if row['channelId'] != string_fillna:
        channel_id = 'channel_' + row['channelId']
        Channel = URIRef(SG[channel_id])
        g.add((Channel, RDF.type, SG.YouTubeChannel))
        
        # data properties
        g.add((Channel, SG['channelName'], Literal(row['originalChannel'], datatype=XSD.string)))
        if row['channelDescription'] != string_fillna:
            g.add((Channel, SG['channelDescription'], Literal(row['channelDescription'], datatype=XSD.string)))
        g.add((Channel, SG['channelViewCount'], Literal(row['viewCount'], datatype=XSD.integer)))
        g.add((Channel, SG['channelSubscribersCount'], Literal(row['subscriberCount'], datatype=XSD.integer)))
        g.add((Channel, SG['channelVideoCount'], Literal(row['videoCount'], datatype=XSD.integer)))
        
        # obj properties
        published_videos = dataset[dataset['Channel'] == row['originalChannel']]['Url_youtube']
        for video_url in published_videos:
                Video = URIRef('video_' + SG[video_url])
                g.add((Channel, SG['publishes'], Video))

CPU times: user 9.39 s, sys: 69.9 ms, total: 9.46 s
Wall time: 9.46 s


In [22]:
%%time
g = write_and_empty_graph(g, 'rdflib_youtube.ttl')

CPU times: user 4.7 s, sys: 99.8 ms, total: 4.8 s
Wall time: 4.83 s


### SpotifyTrack + isRelatedTo

In [23]:
%%time
for index, row in dataset.iterrows():
    track_uri = 'track_' + row['Uri']
    Track = URIRef(SG[track_uri])
    g.add((Track, RDF.type, SG.SpotifyTrack))
    
    # data properties
    g.add((Track, SG['trackName'], Literal(row['Track'], datatype=XSD.string)))
    if row['Duration_ms'] != numeric_fillna:
        g.add((Track, SG['trackAcousticness'], Literal(row['Acousticness'], datatype=XSD.float)))
        g.add((Track, SG['trackDanceability'], Literal(row['Danceability'], datatype=XSD.float)))
        g.add((Track, SG['trackDuration'], Literal(row['Duration_ms'], datatype=XSD.integer)))
        g.add((Track, SG['trackEnergy'], Literal(row['Energy'], datatype=XSD.float)))
        g.add((Track, SG['trackInstrumentalness'], Literal(row['Instrumentalness'], datatype=XSD.float)))    
        g.add((Track, SG['trackKey'], Literal(row['Key'], datatype=XSD.integer)))
        g.add((Track, SG['trackLiveness'], Literal(row['Liveness'], datatype=XSD.float)))
        g.add((Track, SG['trackLoudness'], Literal(row['Loudness'], datatype=XSD.float)))
        g.add((Track, SG['trackSpeechiness'], Literal(row['Speechiness'], datatype=XSD.float)))
        g.add((Track, SG['trackTempo'], Literal(row['Tempo'], datatype=XSD.float)))
        g.add((Track, SG['trackValence'], Literal(row['Valence'], datatype=XSD.float)))
    if row['Stream'] != numeric_fillna:
        g.add((Track, SG['trackStreams'], Literal(row['Stream'], datatype=XSD.integer)))
    
    # object properties
    # isRelatedTo
    if row['Url_youtube'] != string_fillna:
        video_uri = 'video_' + row['Url_youtube']
        Video = URIRef(SG[video_uri])
        g.add((Track, SG['isRelatedTo'], Video))

CPU times: user 8.98 s, sys: 144 ms, total: 9.13 s
Wall time: 9.13 s


### SpotifyAlbum + containsTrack + isAvailableIn

In [24]:
%%time
for index, row in spotify_album.iterrows():
    album_uri = 'album_' + row['Id']
    Album = URIRef(SG[album_uri])
    g.add((Album, RDF.type, SG.SpotifyAlbum))
    
    # data prop
    g.add((Album, SG['albumName'], Literal(row['Album'], datatype=XSD.string)))
    if row['Release_date'] != '0000':
        g.add((Album, SG['albumReleaseDate'], Literal(row['Release_date'], datatype=XSD.date)))
    g.add((Album, SG['albumTotalTracksNum'], Literal(row['Total_tracks'], datatype=XSD.integer)))
    g.add((Album, SG['albumType'], Literal(dataset.at[index, 'Album_type'], datatype=XSD.string))) # TODO check enum datatype
    
    # obj prop
    # isAvailableIn
    market_list = ast.literal_eval(row['Available_market']) # converts the string representation of a list into an object of type list
    for market in market_list:
        Country = URIRef(CNS[market.lower()])
        g.add((Album, SG['isAvailableIn'], Country))
        
    # containsTrack
    Track = URIRef(SG['track_' + dataset.at[index, 'Uri']])
    g.add((Album, SG['containsTrack'], Track))

CPU times: user 38.4 s, sys: 850 ms, total: 39.3 s
Wall time: 39.2 s


In [25]:
%%time
g = write_and_empty_graph(g, 'rdflib_spotify_album_track.ttl')

CPU times: user 33.2 s, sys: 140 ms, total: 33.3 s
Wall time: 33.4 s


### Artist + hasNationality + Genre + hasGenre + composes + writes + performsIn + hasOfficialChannel

In [26]:
%%time
for index, row in wikidata_artists.iterrows():
    row_spotify_artist_info = spotify_artist_info.iloc[index] # one row
    rows_dataset = dataset[dataset['Url_spotify'] == row['Url_spotify']] # 10 rows
    
    artist_uri = "artist_" + row['Url_spotify']
    Artist = URIRef(SG[artist_uri])
    g.add((Artist, RDF.type, SG.SpotifyArtist))
    
    # data properties
    g.add((Artist, SG['artistFollowersNum'], Literal(row_spotify_artist_info['Followers'], datatype=XSD.integer)))
    g.add((Artist, SG['artistName'], Literal(row['Artist'], datatype=XSD.string)))
    g.add((Artist, SG['artistPopularity'], Literal(row_spotify_artist_info['Popularity'], datatype=XSD.integer)))
    if row['websiteLabel'] != '_':
        g.add((Artist, SG['artistWebsite'], Literal(row['websiteLabel'], datatype=XSD.string)))
    if row['start'] != '_':
        g.add((Artist, SG['startWorkingPeriod'], Literal(row['start'], datatype=XSD.gYear)))
    if row['end'] != '_':
        g.add((Artist, SG['endWorkingPeriod'], Literal(row['end'], datatype=XSD.gYear)))
    if row['dissolved'] != '_':
        g.add((Artist, SG['dissolvedIn'], Literal(row['dissolved'], datatype=XSD.gYear)))
    
    # obj prop
    # hasNationality
    if row['country_codes'] != '_':
        cc_list = row['country_codes'].split('+')
        for cc in cc_list:
            Country = URIRef(CNS[cc.lower()])
            g.add((Artist, SG['hasNationality'], Country))
    
    # Genre + hasGenre
    artist_genre_list = ast.literal_eval(row_spotify_artist_info['Genres'])
    for genre_name in artist_genre_list:
        genre_uri = 'genre_' + re.sub('[^A-Za-z0-9\s]', '', genre_name).replace(' ', '_')
        Genre = URIRef(SG[genre_uri])
        g.add((Genre, RDF.type, SG.Genre))
        g.add((Artist, SG['hasGenre'], Genre))
     
    for index2, row2 in rows_dataset.iterrows():
        # composes
        album_uri = 'album_' + spotify_album.iloc[index2]['Id']
        Album = URIRef(SG[album_uri])
        g.add((Artist, SG['composes'], Album))
        
        # writes
        track_uri = 'track_' + row2['Uri']
        Track = URIRef(SG[track_uri])
        g.add((Artist, SG['writes'], Track))
        
        # performsIn
        if row2['Url_youtube'] != string_fillna:
            video_uri = 'video_' + row2['Url_youtube']
            Video = URIRef(SG[video_uri])
            g.add((Artist, SG['performsIn'], Video))
        
        # hasOfficialChannel
        if row2['Channel'] != string_fillna and row2['official_video']:
            yt_channel_row = youtube_api_channels[youtube_api_channels['originalChannel'] == row2['Channel']].iloc[0]
            
            channel_uri = 'channel_' + yt_channel_row['channelId']
            Channel = URIRef(SG[channel_uri])
            
            g.add((Artist, SG['hasOfficialChannel'], Channel))

CPU times: user 21.3 s, sys: 130 ms, total: 21.5 s
Wall time: 21.5 s


In [27]:
%%time
g = write_and_empty_graph(g, 'rdflib_spotify_artist_genre.ttl')

CPU times: user 1.68 s, sys: 10 ms, total: 1.69 s
Wall time: 1.69 s


### Awards + hasReceived

In [28]:
%%time
for index, row in wikidata_awards_statements.iterrows():
    award_id = 'award_' + str(index)
    
    Award = URIRef(SG[award_id])
    Artist = URIRef(SG['artist_' + row['artist_spotify_id']])
    
    award_data = wikidata_awards[wikidata_awards['award_id'] == row['award_id']].iloc[0]
    award_class_year = award_data['award_class'] + (row['award_year'] if row['award_year'] != '_' else '')
    
    g.add((Award, RDF.type, SG[award_class_year]))
    
    # data properties
    g.add((Award, SG['awardName'], Literal(award_data['award_type'], datatype=XSD.string)))
    if award_data['award_category'] != '_':
        g.add((Award, SG['awardCategory'], Literal(award_data['award_category'], datatype=XSD.string)))
    if row['award_year'] != '_':
        g.add((Award, SG['awardYear'], Literal(row['award_year'], datatype=XSD.gYear)))
    
    # object properties
    # hasReceived
    g.add((Artist, SG['hasReceived'], Award))

CPU times: user 450 ms, sys: 20.1 ms, total: 470 ms
Wall time: 460 ms


In [29]:
%%time
g = write_and_empty_graph(g, 'rdflib_awards.ttl')

CPU times: user 96.7 ms, sys: 10.1 ms, total: 107 ms
Wall time: 108 ms
